In [1]:
# Import torch & Check CUDA availability
import torch

print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.current_device())

True
1
0


In [2]:
# Get CUDA device name
print(torch.cuda.device(0))
print(torch.cuda.get_device_name(0))

NVIDIA A100 80GB PCIe


#### Import Graph500_Scale24_EdgeFactor64

In [3]:
# Import dataset from saved PyTorch dataset
data = torch.load("/dfs6/pub/seminl1/graph500/graph500_scale24_ef64.pt")

In [4]:
# Transform dense edge index to sparse adjacency matrix
import torch_geometric.transforms as T

data = T.ToSparseTensor()(data)
data = data.pin_memory()

#### Graph Information

In [6]:
# Print first element
print(f'Graph: {data}')

Graph: Data(x=[16777216, 64], y=[16777216], train_mask=[16777216], adj_t=[16777216, 16777216, nnz=1044447191])


In [7]:
# Node feature matrix information
print(f'x = {data.x.shape}')
print(data.x)

x = torch.Size([16777216, 64])
tensor([[ 1.7491,  1.0508, -1.1073,  ..., -1.6631,  0.4232, -1.6230],
        [ 1.4225, -1.3486,  0.0630,  ..., -0.3219, -1.0920, -0.3329],
        [ 0.6128, -1.0707, -1.4233,  ...,  0.4200,  0.1730,  1.0194],
        ...,
        [ 0.5497, -0.4471,  1.0664,  ...,  0.7018,  0.1765, -1.6260],
        [ 2.3088,  1.4693, -1.4589,  ..., -2.0359,  2.3048,  1.3808],
        [-0.1230,  1.0694,  2.1998,  ..., -1.5174,  0.5692, -1.5635]])


In [8]:
# Adjacency matrix for the edges
print(data.adj_t)

SparseTensor(row=tensor([       0,        0,        0,  ..., 16777215, 16777215, 16777215]),
             col=tensor([       0,        1,        2,  ..., 16147194, 16285027, 16745422]),
             size=(16777216, 16777216), nnz=1044447191, density=0.00%)


In [9]:
# Ground-truth labels
print(f'y = {data.y.shape}')
print(data.y)

y = torch.Size([16777216])
tensor([59, 15,  4,  ..., 61, 27, 59])


In [10]:
# Train mask
print(f'train_mask = {data.train_mask.shape}')
print(data.train_mask)

train_mask = torch.Size([16777216])
tensor([False,  True, False,  ..., False,  True, False])


#### Sigle-layer GCN

In [11]:
# Create a simple GCN with only one GCN layer
import torch.nn.functional as F

from torch.nn import Linear
from torch_geometric.nn import GCNConv

class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.gcn1 = GCNConv(64, 64)
        self.optimizer = torch.optim.Adam(self.parameters(),
                                          lr=0.01,
                                          weight_decay=5e-4)

    def forward(self, x, adj_t):
        x = self.gcn1(x, adj_t)
        z = F.log_softmax(x, dim=1)
        return x, z

#### Do use mini-batch

In [12]:
def accuracy(pred_y, y):
    """Calculate accuracy."""
    return ((pred_y == y).sum() / len(y)).item()

def train(model, train_loader):
    """Train a GNN model and return the trained model."""
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = model.optimizer
    epochs = 100

    model.train()
    for epoch in range(epochs):
        #total_loss = 0
        #acc = 0
        #val_loss = 0
        #val_acc = 0
        
        # Train on batches
        for batch in train_loader:
            batch = batch.to('cuda:0', non_blocking=True)
            optimizer.zero_grad()
            h, out = model(batch.x, batch.adj_t)
            #loss = criterion(out[batch.train_mask], batch.y[batch.train_mask])
            #total_loss += loss
            #acc += accuracy(out[batch.train_mask].argmax(dim=1), batch.y[batch.train_mask])
            #loss.backward()
            optimizer.step()
        
        # Print metrics every 10 epochs
        if(epoch % 10 == 0):
            print('Epoch:', epoch)
            #print(f'Epoch {epoch:>3} | Train Loss: {total_loss/len(train_loader):.3f} | Train Acc: '
            #      f'{acc/len(train_loader)*100:>6.2f}%')

    return model, h, out

In [14]:
from torch_geometric.loader import NeighborLoader
    
start = torch.cuda.Event(enable_timing=True)
end = torch.cuda.Event(enable_timing=True)

# Create GCN model
gcn = GCN()
print(gcn)
print()

# Create batches with neighbor sampling
start.record()
train_loader = NeighborLoader(
    data,
    num_neighbors=[-1],
    batch_size=131072,
    pin_memory=True,
    #num_workers=8,
    #prefetch_factor=1,
)

# Train
gcn_model, gcn_output, final_output = train(gcn.to('cuda:0'), train_loader)
end.record()
torch.cuda.synchronize()
elapsed_time = start.elapsed_time(end)
print('Elapsed Time (100 Epochs):', elapsed_time*0.001, 'seconds')

GCN(
  (gcn1): GCNConv(64, 64)
)

Epoch: 0
Epoch: 10
Epoch: 20
Epoch: 30
Epoch: 40
Epoch: 50
Epoch: 60
Epoch: 70
Epoch: 80
Epoch: 90
Epoch: 100
